### https://www.analyticsvidhya.com/blog/2017/09/machine-learning-models-as-apis-using-flask/

### https://github.com/harshalsonioo1/EndToEndBlog

### https://github.com/ashukumar27/Python_ML_Package

In [2]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
#from Custom_PreProcessing import Custom_PreProcessing
import warnings

warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('data/training.csv')

In [4]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


### EDA

In [5]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [6]:
# Unique value of numeric column
for col in df.columns:
    if df[col].dtype != object:
        print(col)
        print('*'*len(col))
        print(df[col].unique())

ApplicantIncome
***************
[ 5849  4583  3000  2583  6000  5417  2333  3036  4006 12841  3200  2500
  3073  1853  1299  4950  3596  3510  4887  2600  7660  5955  3365  3717
  9560  2799  4226  1442  3750  4166  3167  4692  3500 12500  2275  1828
  3667  3748  3600  1800  2400  3941  4695  3410  5649  5821  2645  4000
  1928  3086  4230  4616 11500  2708  2132  3366  8080  3357  3029  2609
  4945  5726 10750  7100  4300  3208  1875  4755  5266  1000  3333  3846
  2395  1378  3988  2366  8566  5695  2958  6250  3273  4133  3620  6782
  2484  1977  4188  1759  4288  4843 13650  4652  3816  3052 11417  7333
  3800  2071  5316  2929  3572  7451  5050 14583  2214  5568 10408  5667
  2137  2957  3692 23803  3865 10513  6080 20166  2014  2718  3459  4895
  3316 14999  4200  5042  6950  2698 11757  2330 14866  1538 10000  4860
  6277  2577  9166  2281  3254 39999  9538  2980  1863  7933  3089  4167
  9323  3707  2439  2237  8000  1820 51763  3522  5708  4344  3497  2045
  5516  6400  1916 

In [7]:
for col in df.columns:
    if df[col].dtype == object:
        print(col)
        print('*'*len(col))
        print(df[col].unique())

Loan_ID
*******
['LP001002' 'LP001003' 'LP001005' 'LP001006' 'LP001008' 'LP001011'
 'LP001013' 'LP001014' 'LP001018' 'LP001020' 'LP001024' 'LP001027'
 'LP001028' 'LP001029' 'LP001030' 'LP001032' 'LP001034' 'LP001036'
 'LP001038' 'LP001041' 'LP001043' 'LP001046' 'LP001047' 'LP001050'
 'LP001052' 'LP001066' 'LP001068' 'LP001073' 'LP001086' 'LP001087'
 'LP001091' 'LP001095' 'LP001097' 'LP001098' 'LP001100' 'LP001106'
 'LP001109' 'LP001112' 'LP001114' 'LP001116' 'LP001119' 'LP001120'
 'LP001123' 'LP001131' 'LP001136' 'LP001137' 'LP001138' 'LP001144'
 'LP001146' 'LP001151' 'LP001155' 'LP001157' 'LP001164' 'LP001179'
 'LP001186' 'LP001194' 'LP001195' 'LP001197' 'LP001198' 'LP001199'
 'LP001205' 'LP001206' 'LP001207' 'LP001213' 'LP001222' 'LP001225'
 'LP001228' 'LP001233' 'LP001238' 'LP001241' 'LP001243' 'LP001245'
 'LP001248' 'LP001250' 'LP001253' 'LP001255' 'LP001256' 'LP001259'
 'LP001263' 'LP001264' 'LP001265' 'LP001266' 'LP001267' 'LP001273'
 'LP001275' 'LP001279' 'LP001280' 'LP001282' '

In [8]:
df['Gender'].value_counts().sort_values(ascending=False).index[0]

'Male'

In [9]:
pred_var = ['Gender','Married','Dependents','Education','Self_Employed','ApplicantIncome','CoapplicantIncome',
            'LoanAmount','Loan_Amount_Term','Credit_History','Property_Area']


In [10]:
from sklearn.model_selection import train_test_split, GridSearchCV

X_train,X_test,y_train,y_test = train_test_split(df[pred_var],df['Loan_Status'],test_size=0.25,random_state=42)

In [91]:
print(type(X_train))

<class 'pandas.core.frame.DataFrame'>


In [11]:
y_train = y_train.replace({'Y':1, 'N':0})
y_test = y_test.replace({'Y':1, 'N':0})

In [12]:
y_train.head()

92     1
304    1
68     1
15     1
211    0
Name: Loan_Status, dtype: int64

In [95]:
from sklearn.base import BaseEstimator, TransformerMixin

class Custom_PreProcessing(BaseEstimator, TransformerMixin):
    """Custom Pre-Processing estimator for our use-case
    """
    def __init__(self):
        pass
    
    def transform(self, df):
        """Regular transform() that is a help for training, validation & testing datasets
           (NOTE: The operations performed here are the ones that we did prior to this cell)
        """
        pred_var = ['Gender','Married','Dependents','Education','Self_Employed','ApplicantIncome',
                    'CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Property_Area']
        df = df[pred_var]
        
        # Fill Missing Value object datatype
        df['Dependents'] = df['Dependents'].fillna(0)
        df['Married'] = df['Married'].fillna('No')
        df['Gender'] = df['Gender'].fillna(df['Gender'].value_counts().sort_values(ascending=False).index[0])
        #df['Gender'] = df['Gender'].fillna('Male')
        df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna(self.term_mean_)
        df['LoanAmount'] = df['LoanAmount'].fillna(self.amt_mean_)
        df['Self_Employed'] = df['Self_Employed'].fillna('No')
        df['Credit_History'] = df['Credit_History'].fillna(1)
        
        gender = {'Female':0,'Male':1}
        married = {'No':0,'Yes':1}
        education = {'Graduate':0,'Not Graduate':1}
        self_employed = {'No':0,'Yes':1}
        property_area = {'Urban':0,'Rural':1,'Semiurban':2}
        dependent_values = {'3+': 3, '0': 0, '2': 2, '1': 1}
        print('Transform - before replace')
        df.replace({'Gender':gender, 'Married':married, 'Education':education,
                    'Dependents': dependent_values, 'Self_Employed':self_employed,
                    'Property_Area':property_area},inplace=True)
       # print('df.head() : ', df.head())

        return df

        
    def fit(self, df, y=None, **fit_params):
        """Fitting the Training dataset & calculating the required values from train
           e.g: We will need the mean of X_train['Loan_Amount_Term'] that will be used in
                transformation of X_test
        """
        print('Inside Fit')
        self.term_mean_ = df['Loan_Amount_Term'].mean()
        self.amt_mean_ = df['LoanAmount'].mean()
        return self

In [39]:
pipe = make_pipeline(Custom_PreProcessing(),
                    RandomForestClassifier())

In [40]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('custom_preprocessing', Custom_PreProcessing()),
                ('randomforestclassifier', RandomForestClassifier())])

In [41]:
# Key(classifier menioned in pipeline) followed by double undersore(__) and then property
param_grid = {"randomforestclassifier__n_estimators" : [10, 20, 30],
             "randomforestclassifier__max_depth" : [None, 6, 8, 10],
             "randomforestclassifier__max_leaf_nodes": [None, 5, 10, 20], 
             "randomforestclassifier__min_impurity_split": [0.1, 0.2, 0.3]}

In [42]:
grid = GridSearchCV(pipe, param_grid=param_grid, cv=3)

In [43]:
grid.fit(X_train,y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('custom_preprocessing',
                                        Custom_PreProcessing()),
                                       ('randomforestclassifier',
                                        RandomForestClassifier())]),
             param_grid={'randomforestclassifier__max_depth': [None, 6, 8, 10],
                         'randomforestclassifier__max_leaf_nodes': [None, 5, 10,
                                                                    20],
                         'randomforestclassifier__min_impurity_split': [0.1,
                                                                        0.2,
                                                                        0.3],
                         'randomforestclassifier__n_estimators': [10, 20, 30]})

In [44]:
print("Best parameters: {}".format(grid.best_params_))

Best parameters: {'randomforestclassifier__max_depth': 6, 'randomforestclassifier__max_leaf_nodes': 10, 'randomforestclassifier__min_impurity_split': 0.2, 'randomforestclassifier__n_estimators': 10}


In [45]:
print("Validation set score: {:.2f}".format(grid.score(X_test, y_test)))

Validation set score: 0.77


In [46]:
grid.best_estimator_

Pipeline(steps=[('custom_preprocessing', Custom_PreProcessing()),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=6, max_leaf_nodes=10,
                                        min_impurity_split=0.2,
                                        n_estimators=10))])

In [47]:
predicted = grid.predict(X_test)

In [48]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
def classification_metrics(y_test,predicted):
    print('accuracy_score :\n', accuracy_score(y_test,predicted))
    print('confusion_matrix :\n', confusion_matrix(y_test,predicted))
    print('classification_report :\n', classification_report(y_test,predicted))

In [49]:
classification_metrics(y_test,predicted)

accuracy_score :
 0.7727272727272727
confusion_matrix :
 [[21 33]
 [ 2 98]]
classification_report :
               precision    recall  f1-score   support

           0       0.91      0.39      0.55        54
           1       0.75      0.98      0.85       100

    accuracy                           0.77       154
   macro avg       0.83      0.68      0.70       154
weighted avg       0.81      0.77      0.74       154



### Load test set

In [50]:
test_df = pd.read_csv('data/test.csv', encoding="utf-8-sig")
test_df = test_df.head()

In [51]:
test_df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [52]:
grid.predict(test_df)

array([1, 1, 1, 1, 1])

### Replace best estimator in Pipeline after hyperparameter tuning

In [53]:
pipe

Pipeline(steps=[('custom_preprocessing', Custom_PreProcessing()),
                ('randomforestclassifier', RandomForestClassifier())])

In [55]:
pipe.steps.pop(1)

('randomforestclassifier', RandomForestClassifier())

In [56]:
pipe

Pipeline(steps=[('custom_preprocessing', Custom_PreProcessing())])

In [57]:
#Replacing randomforestclassifier with best estimator
pipe.steps.insert(1,['randomforestclassifier',grid.best_estimator_])

In [58]:
pipe

Pipeline(steps=[('custom_preprocessing', Custom_PreProcessing()),
                ['randomforestclassifier',
                 Pipeline(steps=[('custom_preprocessing',
                                  Custom_PreProcessing()),
                                 ('randomforestclassifier',
                                  RandomForestClassifier(max_depth=6,
                                                         max_leaf_nodes=10,
                                                         min_impurity_split=0.2,
                                                         n_estimators=10))])]])

### Saving Pipeline

In [59]:
import pickle

# open a file, where you ant to store the data
file = open('car_price_prediction_pipeline.pkl', 'wb')
pickle.dump(pipe, file)

### Loading Pipeline & predicting it

In [61]:
file = open('car_price_prediction_pipeline.pkl', 'rb')
pipeline = pickle.load(file)

In [62]:
pipeline

Pipeline(steps=[('custom_preprocessing', Custom_PreProcessing()),
                ['randomforestclassifier',
                 Pipeline(steps=[('custom_preprocessing',
                                  Custom_PreProcessing()),
                                 ('randomforestclassifier',
                                  RandomForestClassifier(max_depth=6,
                                                         max_leaf_nodes=10,
                                                         min_impurity_split=0.2,
                                                         n_estimators=10))])]])

In [87]:
#predict = pipeline.predict(['Male','Yes',0,'Not Graduate','No',2583,2358.0,120.0,360.0,1.0,'Urban','Y'])
predict = pipeline.predict([np.array(list({'Gender':'Male','Married':'Yes','Dependents':0,'Education':'Not Graduate','Self_Employed':'No','ApplicantIncome':2583,
  'CoapplicantIncome':2358.0,'LoanAmount':120.0,'Loan_Amount_Term':360.0,'Credit_History':1.0,'Property_Area':'Urban'}))])
input_data = {'Gender':'Male','Married':'Yes','Dependents':0,'Education':'Not Graduate','Self_Employed':'No','ApplicantIncome':2583,
  'CoapplicantIncome':2358.0,'LoanAmount':120.0,'Loan_Amount_Term':360.0,'Credit_History':1.0,'Property_Area':'Urban'}

data = pd.read_json(input_data)
data

<IPython.core.display.Javascript object>

TypeError: list indices must be integers or slices, not list

In [93]:
input_data = {'Gender':'Male','Married':'Yes','Dependents':0,'Education':'Not Graduate','Self_Employed':'No','ApplicantIncome':2583,
  'CoapplicantIncome':2358.0,'LoanAmount':120.0,'Loan_Amount_Term':360.0,'Credit_History':1.0,'Property_Area':'Urban'}

df1= pd.read_json(input_data)
df1

ValueError: Invalid file path or buffer object type: <class 'dict'>

In [90]:
print(type(df.iloc[[77]]))
df.iloc[[77]]

<class 'pandas.core.frame.DataFrame'>


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
77,LP001259,Male,Yes,1,Graduate,Yes,1000,3022.0,110.0,360.0,1.0,Urban,N


In [74]:
df.iloc[[77]][pred_var]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
77,Male,Yes,1,Graduate,Yes,1000,3022.0,110.0,360.0,1.0,Urban


In [96]:
predict = pipeline.predict(test_df)
predict

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'